In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.max_columns", None)

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

#### First loop thru each page and get a list of URLs

In [5]:
base_url = "https://freida.ama-assn.org/search/list?spec=42931&page="

In [6]:
urls = []
for r in range(1, 13):
    urls.append(base_url + str(r))

In [7]:
urls

['https://freida.ama-assn.org/search/list?spec=42931&page=1',
 'https://freida.ama-assn.org/search/list?spec=42931&page=2',
 'https://freida.ama-assn.org/search/list?spec=42931&page=3',
 'https://freida.ama-assn.org/search/list?spec=42931&page=4',
 'https://freida.ama-assn.org/search/list?spec=42931&page=5',
 'https://freida.ama-assn.org/search/list?spec=42931&page=6',
 'https://freida.ama-assn.org/search/list?spec=42931&page=7',
 'https://freida.ama-assn.org/search/list?spec=42931&page=8',
 'https://freida.ama-assn.org/search/list?spec=42931&page=9',
 'https://freida.ama-assn.org/search/list?spec=42931&page=10',
 'https://freida.ama-assn.org/search/list?spec=42931&page=11',
 'https://freida.ama-assn.org/search/list?spec=42931&page=12']

#### Grab the markup from the pages and put each in a list

In [8]:
pages = []

for url in urls:
    page = requests.get(url)
    pages.append(bs4.BeautifulSoup(page.text))

#### Create a list of facility cards from each page

In [9]:
soups = []

for p in pages:
    soups.append(p.find_all("div", class_="search-result-card__main ng-star-inserted"))

#### Loop over the cards from each page and their elements, then extract features

In [10]:
elements = []

for soup in soups:
    for s in soup:
        dict_data = {
            "id": s.find("a", class_="search-result-card__title")["href"],
            "headline": s.find("h4").text,
            "footer": s.find("div", class_=None).text,
        }
        elements.append(dict_data)

#### Turn the list of dicts into a dataframe

In [11]:
df = pd.DataFrame(elements)

#### How many places? 

In [12]:
len(df)

298

#### Cities and states

In [13]:
df[["city", "state"]] = df["footer"].str.split(", ", expand=True)

In [14]:
df.drop("footer", axis=1, inplace=True)

---

In [15]:
df.head()

,id,headline,city,state
0,/program/2200111018,University of Alabama Medical Center Program,Birmingham,AL
1,/program/2200121020,USA Health Program,Mobile,AL
2,/program/2200411026,University of Arkansas for Medical Sciences (U...,Little Rock,AR
3,/program/2200321024,University of Arizona College of Medicine-Phoe...,Phoenix,AZ
4,/program/2200321328,Creighton University School of Medicine (Phoen...,Phoenix,AZ


In [16]:
df.tail()

,id,headline,city,state
293,/program/2205631307,Medical College of Wisconsin Affiliated Hospit...,Milwaukee,WI
294,/program/2205612308,Aurora Health Care Program,Milwaukee,WI
295,/program/2205511303,Charleston Area Medical Center/West Virginia U...,Charleston,WV
296,/program/2205521344,Marshall University School of Medicine Program,Huntington,WV
297,/program/2205511304,West Virginia University Program,Morgantown,WV


---

## Export

In [17]:
df.to_csv("data/processed/facilities_list_scraped.csv", index=False)